In [89]:
import sys

import numpy as np
import pandas as pd
import datetime
import time

In [2]:
phones = ['Iphone', 'samsung', 'nokia']
for index, phone in enumerate(phones):
    print(index, phone)

0 Iphone
1 samsung
2 nokia


In [90]:
import spotipy
import spotipy.util as util

In [91]:
client_id = '073875312eef4884b4300f8b45747490'
client_secret = 'fe10794589bd44ee8c47fc9867330b58'
redirect_uri = 'https://www.google.com/'
scope = 'user-library-read' 
username = 'nguyen'

In [92]:
import os
from json.decoder import JSONDecodeError


try:
    token = util.prompt_for_user_token(username, scope, client_id, client_secret, redirect_uri)
except (AttributeError, JSONDecodeError):
    os.remove(f".cache-{username}")
    token = util.prompt_for_user_token(username, scope, client_id, client_secret, redirect_uri)

In [93]:
spotify = spotipy.Spotify(auth=token)

In [94]:
# timing function
def timefunc(f):
    def f_timer(*args, **kwargs):
        start = time.time()
        result = f(*args, **kwargs)
        end = time.time()
        print (f.__name__, 'took', end - start, 'seconds')
        return result
    return f_timer

In [95]:

@timefunc
# get categories 
def get_categories():
    category_ids = []
    for i in spotify.categories(limit = 50)['categories']['items']:
        category_ids.append(i.get('id'))
    return category_ids

@timefunc
# get playlists from list of categories
def get_playlists(categories):
    playlist_ids = []
    for i in categories:
        for j in spotify.category_playlists(i, limit = 50)['playlists']['items']:
            playlist_ids.append(j.get('id'))
    return playlist_ids

# get song ids from list of playlist ids
@timefunc
def get_songs(playlists):
    song_ids = []
    for i in playlists:
        try:
            for j in spotify.user_playlist('spotify', i)['tracks']['items']:
                song_ids.append(j['track']['id'])
        except:
            pass
        time.sleep(.1)
    return song_ids

In [96]:
categories = get_categories()


get_categories took 0.2685098648071289 seconds


In [97]:
playlists = get_playlists(categories)

get_playlists took 14.338891983032227 seconds


In [98]:

songs = get_songs(playlists)

get_songs took 605.2927558422089 seconds


In [99]:
songs_df = pd.DataFrame(list(set(songs)))


In [100]:
print (len(playlists))
print (len(songs_df))

1154
49846


In [101]:
songs_df.columns = ['id']

In [106]:
def chunks(l, n):
    n = max(1, n)
    return [l[i:i+n] for i in range(0, len(l), n)]

In [116]:
@timefunc
def complete_songs_df(df):
    spotify_title_artists = []

    for j in chunks(list(df.id),50):
        try:
            for i in spotify.tracks(j)['tracks']:
                try:
                    title_artist = {}
                    primary_artist = i['artists'][0]['name'].encode('latin-1', 'replace')
                    name = i['name'].encode('latin-1', 'replace')
                    popularity = i['popularity']
                    explicit = i['explicit']
                    ID = i['id']
                    
                    title_artist['id'] = ID
                    title_artist['title'] = name
                    title_artist['artist'] = primary_artist
                    title_artist['popularity'] = popularity
                    title_artist['explicit'] = explicit
                    spotify_title_artists.append(title_artist)
                except:
                    pass
        except:
            pass
    
    return spotify_title_artists

In [117]:
songs_plus = complete_songs_df(songs_df)

complete_songs_df took 319.46015214920044 seconds


In [120]:
songs_df_plus = pd.DataFrame(songs_plus)

In [121]:
len(songs_df_plus)


49846

In [127]:
songs_df_plus['artist'].unique()

array([b'Oscar Peterson', b'V??ng Ch\xed Trung', b'Amit Trivedi', ...,
       b'The American Spirit', b'Lord Dying', b'Reddish Blu'],
      dtype=object)

In [80]:
songs_df_plus.to_pickle('./data/songs_df.pkl')

In [81]:
songs_df_plus = pd.read_pickle('data/songs_df.pkl')

In [84]:
def complete_audio_features(df):
    features_list = []
    for j in chunks(list(df.id),50):
        features_add = spotify.audio_features(tracks=j)
        features_list.extend(features_add)
    return features_list

In [92]:

audio_features = complete_audio_features(songs_df_plus)

In [96]:
def make_audio_features_df(audio_features):
    audio_features_df = pd.DataFrame(columns = audio_features[0])
    for i in audio_features:
        audio_features_df = audio_features_df.append(i, ignore_index=True)
    return audio_features_df

In [99]:

audio_features_df = make_audio_features_df(audio_features)

In [223]:
audio_features_df['uri'][3]

'spotify:track:70HWRkitkAKfos500LBk0T'

In [122]:

audio_features_df.to_pickle('data/audio_features.pkl')

In [123]:
audio_features_df = pd.read_pickle('data/audio_features.pkl')


In [124]:

songs = pd.merge(songs_df_plus, audio_features_df, on='id')

In [125]:
songs.drop(['track_href', 'analysis_url', 'type'], axis=1, inplace=True)

In [10]:
songs.head()

NameError: name 'songs' is not defined

In [120]:
songs.to_pickle('spotify_songs.pkl')

In [11]:
audio_features_df = pd.read_pickle('./spotify_songs.pkl')

In [ ]:
# 31765

In [12]:
audio_features_df = 

,artist,explicit,id,popularity,title,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,b'The Midnight',False,2B7uH2eyDBROM2qJG9QE9Z,45,b'Youth (Instrumental)',0.137,0.2220,2,-25.826,1,0.0460,0.926000,0.904000,0.3150,0.1070,168.658,189630,4
1,b'HYOMIN',False,1YX290GRxBTdJ7C0rJBzF0,31,b'Cabinet',0.499,0.9290,10,-1.827,0,0.0936,0.322000,0.000089,0.2010,0.6590,114.497,223453,4
2,b'Calum Scott',False,6EkQmHBoux7iqK4vjfHq4V,63,b'Need To Know',0.728,0.7390,3,-6.950,1,0.1510,0.096800,0.000000,0.0889,0.3110,125.064,167587,4
3,b'Passenger',False,70HWRkitkAKfos500LBk0T,62,b'Simple Song',0.631,0.4020,5,-11.125,1,0.0286,0.616000,0.011400,0.0892,0.5980,78.000,228480,4
4,b'Karma',False,3sTILrfXTqgroX8Wo5ZxAs,31,b'Terrorist',0.745,0.5040,11,-8.151,0,0.0378,0.007320,0.438000,0.0826,0.6870,138.039,327664,4
5,b'Jax Jones',False,5vnLe2h6cHuQdAKShkOxlm,50,b'Harder (with Bebe Rexha)',0.899,0.8420,5,-2.982,0,0.0844,0.086700,0.000001,0.1610,0.7120,104.014,160267,4
6,b'Covet',False,10OAoychwKbKKmizb7rOzf,43,b'Ares',0.341,0.6580,6,-10.166,0,0.0330,0.317000,0.831000,0.3710,0.6290,126.016,350811,1
7,b'Hecate Enthroned',False,4at29LJNxiDPBkXBvVpKKv,17,b'Temples That Breathe',0.308,0.8330,4,-3.895,1,0.0498,0.000006,0.009510,0.0860,0.0948,95.076,275661,3
8,b'Bachata Heightz',False,6DPdMLNGI66ALjWGhYgfRs,49,b'Cuentale',0.776,0.8510,5,-4.304,0,0.0338,0.181000,0.000001,0.3090,0.9010,134.985,220500,4
9,b'Silver Sphere',False,0W3mgC3fJr3f2Bl635yyNP,51,b'waste my time',0.556,0.7430,3,-5.002,1,0.0319,0.004080,0.711000,0.1350,0.1660,94.994,206227,4


In [196]:
artist.keys()

dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track_number', 'type', 'uri'])

In [213]:
# spotify.album('')
urn = 'spotify:track:2B7uH2eyDBROM2qJG9QE9Z'
# sp = spotipy.Spotify()

# spotify.trace = True # turn on tracing
# spotify.trace_out = True # turn on trace out

artist = spotify.track(urn)
print(artist['album']['images'][0]['url'])


https://i.scdn.co/image/7257f173642055e1daa229b38081f43aaae4e5c5


In [216]:
def get_url_tracks(urn):
    artist = spotify.track(urn)
    return artist['album']['images'][0]['url']

In [217]:
def out_of_range(urn):
    urn = 'data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAARQAAAC3CAMAAADkUVG/AAAAbFBMVEX////u7u709PQ/Pz9PT09ISEjY2Nj4+Pj8/PzExMRvb2/BwcFUVFS6urqrq6udnZ18fHzm5uZDQ0NaWlqhoaFkZGTQ0NCVlZXa2tp1dXWzs7NRUVHv7+9hYWGCgoLKysqmpqaKioo1NTUzMzPoUfqYAAAFrUlEQVR4nO2cbZ+iLBTGVSTRJMVnLTV3v/93vDmA09Tezb5am7Hr/6IInH5wDZwDB8jzAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAOD7otSra/BdCAibrNunT4mNavNN4AVjLKyo1clTUZZswxq9HnGufKVk2ntfiZJUG1bp9YjwRG+HYrai+J/syi3dX9fURb3BSHKieKPuC0mmSs6LwZZIzjlrtQTBOSzTsLjovLksQjYFr6vuNqyikJFNlrERwYk1lFExGUXNWAsvUippFfUQyU5B1CXny0ur/O9xoly41IOEHyid5/qlYTGlVWhMbG9sil+YZ4NxeklVt8OKcsnDSDc9MVlDql+W2pZXnF6tKFVo7Yks/O0ruiValDmp2Ug21XmfWAsUpVdbfmCdt4qSuB7iF/H2Fd0SLcohOxkrsooya1ECdrTlimu35EQprzYvOp+2rua2rIaWuBPFdQbFKWFEEaV79G1FCZ1j7goyvlaUerF5QSG3reTWPBFF9LnNG5i2wG74tKXNa4yd2TFPRNFTEjOdFbUxrnaa37DZPJDXG1dya56JEvUl+d2FG+87GWnEElIXqfbeUTzBrh9pFzqIzYi5TKyf+GiXP0fW99pDRS1LlnPavKCemyJUcEva6fvFrQM7KQ9rUSNjUspTUjZvsCIEAAAAAABgL8Sji5d41z+XwdXOI5DPkGfu1jrZ+bEsOB+3rs73QKbTaFNZ+lgmynftKWHngpBOlCY/pznFCfxpKfvl+ZmEHSN5MBQmQGBFubKqO2TspEUZhrKVbzmAtCjRaLqDEcXtEQ6MDE30tsOHB95sQo1GlKS32QmFsIPyLfuJFcXLaeuURPG528cYKGT73qJ0tCVKonSFC0/PFNd/b1H0HEVYUZgL1TYFRLmUlespLmg/0xmDNxfFk8yvtCiXdcO4SgVEEclyNSdU3PS2JC/97qJ4B57QprEqzCGmltNuUFQOL6zZC5HM7Yu13CwI5zAZhjq0i8QlHKq9H3X7Pzrp9v58N6P3T8syRDbvUi1Z9KKKAQAAAACALxE4wvUHUaintOL0cC0h/nQAUAz7u7MQzV9eLA3qWU/yfx/uc/tPexz+r4fCHXBlfz8f7LOHdufZF4U/H1FmxV8PCL+bKDMX7nS176IF5j1q5GxXwn50a7c6xnasaVGCWVox18JOznu5Udhn3lAYB7PYy2A+m71oKeo+5WRNI35c2y3TtK+ZUTCvZpb0LAm8tTDqWV/v5EqHYspTdlsntoElGWrrakInrdnlSWPXbiGpl9jNwilJ9Qc1ko5WlL42Vxt2cSy9ombZS4J+aLZD8w/H4lMQ/yaKY6KTK5M9qdGQCKYwtncJ+37Duv8rzCzEmwtjKUxzg+Ljnx2QSn+I0lK7J3vdJxozVzjZQz4D30FI7sjJnAgTpfck3SaN3TVS/9AYxe5EibomMKLk7qATXdGlQpFaa+Lk/dkk1rWeyHro4aKbvpjmxiMrR/EgippYWB6sKM4lt7UtjNh8KsdxLNnPF6Ur2oHIuAlP95UnjMsZWEZbgfei+Of6ELnhs4qyfIhyVAdN1/38lVJW1pbSGMhh9BoaT1FqhtODKNnZ2AsriruW3E9OlHQ3l04Dvh6/MccJyD9n1Frfela/uBPF3pPzcmNouf2C8+BsSr4Ht2OQfE0FJY0HUQ+0/LO/dqAtDb8TxU7uFDei2OuGA7u4wnWG8vOdT3JbwVyNPtfS+B4xlUrP1cb6TpSYSZqv5Xb4hJImJ1fPFUZ9qOVUblb8g2l+3VyFMnf3u8K6Wj8vOO/9WjrDYqID4sQ5D+eB2t1nXao/me5iQwdBq/+EVz++p3zlKFTnf3rIPRl06pYplPuhnfVrAvziFwAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAOBb8B/fJkUOV0v6DAAAAABJRU5ErkJggg=='
    return urn

In [207]:
songs.head()

,artist,explicit,id,popularity,title,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,uri,duration_ms,time_signature
0,b'The Midnight',False,2B7uH2eyDBROM2qJG9QE9Z,45,b'Youth (Instrumental)',0.137,0.222,2,-25.826,1,0.0460,0.92600,0.904000,0.3150,0.107,168.658,spotify:track:2B7uH2eyDBROM2qJG9QE9Z,189630,4
1,b'HYOMIN',False,1YX290GRxBTdJ7C0rJBzF0,31,b'Cabinet',0.499,0.929,10,-1.827,0,0.0936,0.32200,0.000089,0.2010,0.659,114.497,spotify:track:1YX290GRxBTdJ7C0rJBzF0,223453,4
2,b'Calum Scott',False,6EkQmHBoux7iqK4vjfHq4V,63,b'Need To Know',0.728,0.739,3,-6.950,1,0.1510,0.09680,0.000000,0.0889,0.311,125.064,spotify:track:6EkQmHBoux7iqK4vjfHq4V,167587,4
3,b'Passenger',False,70HWRkitkAKfos500LBk0T,62,b'Simple Song',0.631,0.402,5,-11.125,1,0.0286,0.61600,0.011400,0.0892,0.598,78.000,spotify:track:70HWRkitkAKfos500LBk0T,228480,4
4,b'Karma',False,3sTILrfXTqgroX8Wo5ZxAs,31,b'Terrorist',0.745,0.504,11,-8.151,0,0.0378,0.00732,0.438000,0.0826,0.687,138.039,spotify:track:3sTILrfXTqgroX8Wo5ZxAs,327664,4


In [218]:
try:
  songs['img_url'] = songs['uri'].apply(get_url_tracks)
except IndexError :
  songs['img_url'] = songs['uri'].apply(out_of_range)


In [148]:
object_methods = [method_name for method_name in dir(spotify)
                  if callable(getattr(spotify, method_name))]

In [149]:
object_methods.

['__class__',
 '__delattr__',
 '__dir__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '_auth_headers',
 '_delete',
 '_get',
 '_get_id',
 '_get_uri',
 '_internal_call',
 '_post',
 '_put',
 '_warn',
 'album',
 'album_tracks',
 'albums',
 'artist',
 'artist_albums',
 'artist_related_artists',
 'artist_top_tracks',
 'artists',
 'audio_analysis',
 'audio_features',
 'categories',
 'category_playlists',
 'current_user',
 'current_user_followed_artists',
 'current_user_playlists',
 'current_user_saved_albums',
 'current_user_saved_albums_add',
 'current_user_saved_tracks',
 'current_user_saved_tracks_add',
 'current_user_saved_tracks_contains',
 'current_user_saved_tracks_delete',
 'current_user_top_artists',
 'current_user_top_tracks',
 'featured_playlists',
 'me',
 

In [128]:
lst_artist = pd.read_csv('data.csv', encoding='utf-8')

In [132]:
lst_artist.columns

Index(['Position', 'Track Name', 'Artist', 'Streams', 'URL', 'Date', 'Region'], dtype='object')

In [133]:
lst_artist['Region'].unique()

array(['ec', 'fr', 'ar', 'fi', 'no', 'it', 'lt', 'ph', 'tw', 'nz', 'ee',
       'tr', 'us', 'sv', 'cr', 'de', 'cl', 'jp', 'br', 'hn', 'gt', 'ch',
       'hu', 'ca', 'pe', 'be', 'my', 'dk', 'bo', 'pl', 'at', 'pt', 'se',
       'mx', 'pa', 'uy', 'is', 'es', 'cz', 'ie', 'nl', 'sk', 'co', 'sg',
       'id', 'do', 'lu', 'gb', 'global', 'py', 'au', 'lv', 'gr', 'hk'],
      dtype=object)

In [143]:
artist = lst_artist[(lst_artist['Region'] == 'au') | (lst_artist['Region'] == 'us')]

In [158]:
lst_of_artist = artist['Artist'].unique()
len(lst_of_artist)

725

In [160]:
artist = str(input())
if artist in lst_of_artist:
    print('True')
else:
    print('False')

Michael Jackson
True


In [153]:
with open('artists.txt', 'w') as filehandle:
    for listitem in lst_of_artist:
        filehandle.write('%s\n' % listitem)

In [155]:
artists = []
with open('artists.txt', 'r') as filehandle:
    for line in filehandle:
        # remove linebreak which is the last character of the string
        currentPlace = line[:-1]

        # add item to the list
        artists.append(currentPlace)

In [156]:
places

['Migos',
 'Drake',
 'The Weeknd',
 'The Chainsmokers',
 'Rae Sremmurd',
 'DRAM',
 'Aminé',
 'DJ Snake',
 'Big Sean',
 'Bruno Mars',
 'Machine Gun Kelly',
 '21 Savage',
 'ZAYN',
 'Maroon 5',
 'Rob $tone',
 'J. Cole',
 'Young M.A',
 'Chance The Rapper',
 'Hailee Steinfeld',
 'Ariana Grande',
 'Shawn Mendes',
 'Martin Garrix',
 'Justin Timberlake',
 'Fifth Harmony',
 'Twenty One Pilots',
 'Lil Uzi Vert',
 'Calvin Harris',
 'James Arthur',
 'Young Thug',
 'KYLE',
 'Sia',
 'Rihanna',
 'Major Lazer',
 'Desiigner',
 'Alessia Cara',
 'Starley',
 'Adele',
 'John Legend',
 'Justin Bieber',
 'Zay Hilfigerrr',
 'The Jackson 5',
 'DNCE',
 'Future',
 'Travis Scott',
 'Childish Gambino',
 'Steve Aoki',
 'Michael Jackson',
 'Clean Bandit',
 'Marvin Gaye',
 'Lil Wayne',
 'Shakira',
 'Foster The People',
 'G-Eazy',
 'DJ Esco',
 'Sage The Gemini',
 'Mike Posner',
 'Pharrell Williams',
 'Jon Bellion',
 'Lil Yachty',
 'M.I.A.',
 'David Guetta',
 'Queen',
 'Post Malone',
 'The Vamps',
 'Cashmere Cat',
 'gn